In [5]:
##################################################
# LIBRARIES 
##################################################
%reset -f

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re, sqlite3, pickle, time, datetime, random, sys
# sys.path is a list of absolute path strings
sys.path.append('/home/d/dlr10/Documents/02_Statitics_modelling/0_FunctionsScripts')
import Loading_Data_Functions as load_fn
import FineTuning_Functions as FineTuning
import Learning_Curves_Functions as LearningCurves

pd.options.display.float_format = '{:,.3f}'.format

In [6]:
# https://towardsdatascience.com/minirocket-fast-er-and-accurate-time-series-classification-cdacca2dcbfa
# https://towardsdatascience.com/sktime-a-unified-python-library-for-time-series-machine-learning-3c103c139a55
# https://pypi.org/project/sktime/
# https://arxiv.org/pdf/1302.2277.pdf
# https://github.com/alan-turing-institute/sktime/tree/main/sktime/transformations/panel/rocket

# 1. Load Data

In [7]:
t = time.time()
path = r'/home/d/dlr10/Documents/02_Statitics_modelling/DataSets/'

df_patients_16_18, df_admissions_16_18, df_eobs_16_18 = load_fn.Load_data()
df_patients_19_20, df_admissions_19_20, df_eobs_19_20 = load_fn.Load_data('2019_2020')

X_data_16_18 = pickle.load(open(path + 'df_ts_2016_18.pickle','rb'))
X_data_19_20 = pickle.load(open(path + 'df_ts_2019_20.pickle','rb'))

print("=================================FIRST RECORDED SAMPLE ADMISSION=================================")
print("-------------------------------------- 2016 - 2018 ----------------------------------------------")
print(len(X_data_16_18['admission_id'].unique().tolist()))
display(X_data_16_18.head(3))
print("")
print("-------------------------------------- 2019 - 2020 ----------------------------------------------")
print(len(X_data_19_20['admission_id'].unique().tolist()))
display(X_data_19_20.head(3))

print("Elapsed time:", time.time()-t)

=================================FIRST RECORDED SAMPLE ADMISSION=================================
-------------------------------------- 2016 - 2018 ----------------------------------------------
7657


,admission_id,rr,ews,heart_rate,temperature,sbp,dbp,Oxygen_Saturation,Assisted_O2,Confusion,CREA,UREA,K,GFR,WBC,PLT,HCT,HGB,RBC,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI,no_sample_series,sex,ethnicity,age_at_admin,Comorb_score,Spcfc_Comorb,Mortality
0,30454500,16.000,9.000,143.000,36.400,81.000,49.000,90.000,1.000,1.000,76.000,17.350,4.300,69.000,6.500,201.500,0.250,81.000,3.130,25.800,80.500,5.525,0.640,0.025,0.270,0.020,30.000,120.500,8.000,0,1.000,0.000,72.000,0.000,1.000,0.000
1,30454500,15.000,7.000,115.667,36.500,88.667,62.333,92.667,1.000,1.000,76.000,17.350,4.300,69.000,6.500,201.500,0.250,81.000,3.130,25.800,80.500,5.525,0.640,0.025,0.270,0.020,30.000,120.500,8.000,1,1.000,0.000,72.000,0.000,1.000,0.000
2,30454500,14.000,5.000,88.333,36.600,96.333,75.667,95.333,1.000,1.000,76.000,17.350,4.300,69.000,6.500,201.500,0.250,81.000,3.130,25.800,80.500,5.525,0.640,0.025,0.270,0.020,30.000,120.500,8.000,2,1.000,0.000,72.000,0.000,1.000,0.000



-------------------------------------- 2019 - 2020 ----------------------------------------------
6531


,admission_id,rr,ews,heart_rate,temperature,sbp,dbp,Oxygen_Saturation,Assisted_O2,Confusion,CREA,UREA,K,GFR,WBC,PLT,HCT,HGB,RBC,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI,no_sample_series,sex,ethnicity,age_at_admin,Comorb_score,Spcfc_Comorb,Mortality
0,58578756,20.000,2.000,110.000,38.400,147.000,92.000,97.000,0.000,0.000,95.000,8.700,4.600,75.000,5.600,174.000,0.384,125.000,4.210,29.600,91.000,4.230,0.800,0.030,0.400,0.080,45.000,137.000,12.000,0,-1.000,0.000,59.000,6.000,1.000,0.000
1,58578756,19.619,2.000,108.190,38.343,145.762,90.667,96.905,0.000,0.000,95.000,8.700,4.600,75.000,5.600,174.000,0.384,125.000,4.210,29.600,91.000,4.230,0.800,0.030,0.400,0.080,45.000,137.000,12.000,1,-1.000,0.000,59.000,6.000,1.000,0.000
2,58578756,19.238,2.000,106.381,38.286,144.524,89.333,96.810,0.000,0.000,94.750,8.669,4.599,75.156,5.591,173.385,0.385,125.240,4.219,29.596,91.010,4.201,0.798,0.030,0.398,0.079,44.958,137.073,11.969,2,-1.000,0.000,59.000,6.000,1.000,0.000


Elapsed time: 23.60969042778015


In [8]:
X_data = pd.concat([X_data_16_18, X_data_19_20])
display(X_data.head(4))

,admission_id,rr,ews,heart_rate,temperature,sbp,dbp,Oxygen_Saturation,Assisted_O2,Confusion,CREA,UREA,K,GFR,WBC,PLT,HCT,HGB,RBC,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI,no_sample_series,sex,ethnicity,age_at_admin,Comorb_score,Spcfc_Comorb,Mortality
0,30454500,16.000,9.000,143.000,36.400,81.000,49.000,90.000,1.000,1.000,76.000,17.350,4.300,69.000,6.500,201.500,0.250,81.000,3.130,25.800,80.500,5.525,0.640,0.025,0.270,0.020,30.000,120.500,8.000,0,1.000,0.000,72.000,0.000,1.000,0.000
1,30454500,15.000,7.000,115.667,36.500,88.667,62.333,92.667,1.000,1.000,76.000,17.350,4.300,69.000,6.500,201.500,0.250,81.000,3.130,25.800,80.500,5.525,0.640,0.025,0.270,0.020,30.000,120.500,8.000,1,1.000,0.000,72.000,0.000,1.000,0.000
2,30454500,14.000,5.000,88.333,36.600,96.333,75.667,95.333,1.000,1.000,76.000,17.350,4.300,69.000,6.500,201.500,0.250,81.000,3.130,25.800,80.500,5.525,0.640,0.025,0.270,0.020,30.000,120.500,8.000,2,1.000,0.000,72.000,0.000,1.000,0.000
3,30454500,13.000,3.000,61.000,36.700,104.000,89.000,98.000,1.000,0.000,76.000,17.350,4.300,69.000,6.500,201.500,0.250,81.000,3.130,25.800,80.500,5.525,0.640,0.025,0.270,0.020,30.000,120.500,8.000,3,1.000,0.000,72.000,0.000,1.000,0.000


In [9]:

feat_list = X_data.columns.tolist()
feat_list = feat_list[1:-1]

In [ ]:
len(X_data['admission_id'])

# Fine tuning Hyperparameters
- optimizer
- learning rate

In [21]:

# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import floor
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten

from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import LeakyReLU

from sklearn.metrics import make_scorer, accuracy_score
LeakyReLU = LeakyReLU(alpha=0.1)


from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [149]:
# Load dataset
trainSet = pd.read_csv('train.csv')
# Feature generation: training data
train = trainSet.drop(columns=['Name', 'Ticket', 'Cabin'])
train = train.dropna(axis=0)
train = pd.get_dummies(train)
# train validation split
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=['PassengerId','Survived'], axis=0),
                                                  train['Survived'], test_size=0.2, random_state=111, 
                                                  stratify=train['Survived'])

In [32]:
# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizerD = {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate),
                  'Adadelta':Adadelta(lr=learning_rate),'Adagrad':Adagrad(lr=learning_rate), 
                  'Adamax':Adamax(lr=learning_rate),'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons    = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs     = round(epochs)
    opt        = list(optimizerD.keys())[round(optimizer)]
    
    # Make scorer accuracy
    score_acc = make_scorer(accuracy_score)
    
    def nn_cl_fun():        
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    
    es    = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn    = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [33]:
# Set paramaters
params_nn ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
|  1        |  0.6983   |  5.51     |  335.3    |  54.88    |  0.7716   |  36.58    |  1.044    |
|  2        |  0.7629   |  0.2023   |  536.2    |  39.09    |  0.3443   |  99.16    |  1.664    |
|  3        |  0.6186   |  0.7307   |  735.7    |  69.7     |  0.2815   |  51.96    |  0.8286   |
|  4        |  0.5719   |  0.6656   |  920.6    |  83.52    |  0.8422   |  83.37    |  6.937    |
|  5        |  0.7385   |  5.195    |  851.0    |  53.71    |  0.03717  |  50.87    |  0.7373   |
|  6        |  0.5719   |  7.355    |  758.2    |  65.22    |  0.2815   |  99.86    |  0.9663   |
|  7        |  0.5488   |  5.539    |  588.0    |  52.4     |  0.7306   |  39.05    |  2.804    |
|  8        |  0.5719   |  2.871    |  957.8    |  93.5     |  0.8157   |  13.07    |  6.604    |
|  9        |  0.766

In [34]:
params_nn_  = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu','elu', 'exponential', LeakyReLU,'relu']
optimizerL  = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']

params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs']     = round(params_nn_['epochs'])
params_nn_['neurons']    = round(params_nn_['neurons'])
params_nn_['optimizer']  = optimizerL[round(params_nn_['optimizer'])]
params_nn_

{'activation': 'sigmoid',
 'batch_size': 704,
 'epochs': 92,
 'learning_rate': 0.3463993048863762,
 'neurons': 69,
 'optimizer': 'Nadam'}

# Fine Tuning with Layers

In [38]:

# Create function
def nn_cl_bo2(neurons, activation, optimizer, learning_rate, batch_size, epochs,layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD = {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons = round(neurons)
    activation1 = activationL[round(activation1)]
    activation2 = activationL[round(activation2)]
    optimizer  = list(optimizerD.keys())[round(optimizer)]
    batch_size = round(batch_size)
    epochs     = round(epochs)
    layers1    = round(layers1)
    layers2    = round(layers2)
    
    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation1))
        if normalization > 0.5:
            nn.add(BatchNormalization())
        for i in range(layers1):
            nn.add(Dense(neurons, activation=activation1))
        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            nn.add(Dense(neurons, activation=activation2))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [39]:
params_nn2 ={
    'neurons': (10, 100),
    'activation1':(0, 9),
    'activation2':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6416   |  5.51     |  335.3    |  0.4361   |  0.2308   |  43.63    |  1.298    |  1.045    |  0.426    |  31.48    |  0.3377   |  6.935    |
|  2        |  0.769    |  2.14     |  265.0    |  0.6696   |  0.1864   |  41.94    |  1.932    |  1.237    |  0.08322  |  91.07    |  0.794    |  5.884    |
|  3        |  0.5719   |  7.337    |  992.8    |  0.5773   |  0.2441   |  53.71    |  1.055    |  1.908    |  0.1143   |  83.55    |  0.6977   |  3.957    |
|  4        |  0.5792   |  2.468    |  998.8    |  0.138    |  0.1846   |  58.8     |  1.81     |  2.456    |  0.3296   |  46.05    |  0.319    |  6.631    |
|  5        |  0.769    |  8.268    |  851.1    |  0

In [41]:
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons'] = round(params_nn_['neurons'])

optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
             'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
             'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
             'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
params_nn_['optimizer'] = list(optimizerD.keys())[round(params_nn_['optimizer'])]
params_nn_

{'activation': 'elu',
 'batch_size': 612,
 'dropout': 0.5801259080387516,
 'dropout_rate': 0.14793085825348554,
 'epochs': 79,
 'layers1': 3,
 'layers2': 3,
 'learning_rate': 0.13626453282900686,
 'neurons': 95,
 'normalization': 0.8777167752038778,
 'optimizer': 'Adamax'}

# Our Problem

In [16]:
df_admissions = df_admissions_16_18.copy()
X_data = X_data_16_18.copy()

list_admins      = X_data['admission_id'].unique().tolist()
list_admins_ord  = df_admissions[df_admissions['admission_id'].isin(list_admins)].sort_values(by = 'admission_date_time')['admission_id']

num_train_admins = int(0.6 * len(list_admins_ord))
num_val_admins   = int(0.2 * len(list_admins_ord))
num_test_admins  = len(list_admins_ord) - num_train_admins - num_val_admins

feat_list = X_data.columns.tolist()[1:-1]
####################################################
####################################################
#  2. NORMALISATION OF TRAINING SET
####################################################
raw_data  = X_data[feat_list]
mortality = X_data['Mortality'].tolist()


mean = raw_data[:num_train_admins * 144].mean(axis=0)
std  = raw_data[:num_train_admins * 144].std(axis=0)
std  = [1 if s == 0 else s for s in std]

norm_data = raw_data - mean
norm_data /= std

norm_data = norm_data.drop(columns = 'no_sample_series')

num_samples  = len(X_data['admission_id'].unique().tolist())
num_features = len(norm_data.columns.tolist())

X_train_mio = np.array(norm_data).reshape((num_samples, 144, num_features ))



df          = X_data[['admission_id', 'Mortality']].groupby(by= 'admission_id').mean()
mort_dict   = dict(zip(df.index.tolist(),df['Mortality']))
y_train_mio = np.array([mort_dict[adm] for adm in X_data['admission_id'].unique().tolist()])

print(X_train_mio.shape, y_train_mio.shape)

(7657, 144, 33) (7657,)


In [17]:
print(num_samples)
print(num_features)
print(len(norm_data))

num_samples * num_features * 144

7657
33
1102608


36386064

## 1. Model alone

In [23]:

nn = Sequential()
nn.add(Flatten(input_shape=(144,33)))
nn.add(Dense(16, activation="relu"))
nn.add(Dense(1, activation = "sigmoid"))


nn.compile(optimizer="rmsprop", loss="binary_crossentropy",
              metrics=["accuracy", keras.metrics.AUC(from_logits=True)])
history = nn.fit(X_train_mio, y_train_mio,
                    epochs=4)
                    

Epoch 1/4
240/240 [==============================] - 1s 1ms/step - loss: 0.6313 - accuracy: 0.7584 - auc_1: 0.7001
Epoch 2/4
240/240 [==============================] - 0s 1ms/step - loss: 0.4982 - accuracy: 0.7960 - auc_1: 0.7629
Epoch 3/4
240/240 [==============================] - 0s 1ms/step - loss: 0.4549 - accuracy: 0.8037 - auc_1: 0.7829
Epoch 4/4
240/240 [==============================] - 0s 1ms/step - loss: 0.4221 - accuracy: 0.8125 - auc_1: 0.8009


## 2. Model in function

In [2]:
neurons      = 3
activation   = 'relu'
batch_size   = 200
epochs       = 30
optimizer    = Adam(lr=0.06)
dropout_rate = 0.2

# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

        
        
def nn_cl_fun():        
    nn = Sequenatial()
    nn.add(Flatten(input_dim=(144,33)))
    nn.add(BatchNormalization())
    nn.add(Dense(neurons1, activation=activation1))
    nn.add(Dropout(dropout_rate1, seed=123))
    nn.add(Dense(neurons2, activation=activation2))
    nn.add(Dropout(dropout_rate2, seed=123))

    nn.add(Dense(1, activation='sigmoid'))
    

    nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return nn

es    = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
nn    = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
score = cross_val_score(nn, X_train_mio, y_train_mio, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
score

NameError: name 'Adam' is not defined

## 3. Model in function with variables

In [182]:
neurons    = 3
activation = 'relu'
batch_size = 200
epochs     = 30
opt        = Adam(lr=0.06)

# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

def nn_cl_fun():        
    inputs  = keras.Input(shape=(144,33))
    x       = Flatten()(inputs)
    x       = Dense(neurons, activation=activation)(x)
    outputs = Dense(1, activation = activation)(x)
    nn      = keras.Model(inputs, outputs)

    nn.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
    return nn

es    = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
nn    = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
score = cross_val_score(nn, X_train_mio, y_train_mio, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
score

0.7642561986994626

## 4. Model in Optimization

In [185]:
# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate,  batch_size, epochs ):
    optimizerD = {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate),
                  'Adadelta':Adadelta(lr=learning_rate),'Adagrad':Adagrad(lr=learning_rate), 
                  'Adamax':Adamax(lr=learning_rate),'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons    = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs     = round(epochs)
    opt        = list(optimizerD.keys())[round(optimizer)]
    
    # Make scorer accuracy
    score_acc = make_scorer(accuracy_score)
    
    def nn_cl_fun():        
        inputs  = keras.Input(shape=(144,33))
        x       = Flatten()(inputs)
        x       = Dense(neurons, activation=activation)(x)        
        x       = Dense(neurons, activation=activation)(x)
        outputs = Dense(1, activation = activation)(x)
        nn      = keras.Model(inputs, outputs)

        nn.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])
        return nn
    
    es    = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn    = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train_mio, y_train_mio, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [186]:
# Set paramaters
params_nn ={
    'neurons': (10, 100),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100)
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
|  1        |  0.8006   |  5.51     |  335.3    |  54.88    |  0.7716   |  36.58    |  1.044    |
|  2        |  0.795    |  0.2023   |  536.2    |  39.09    |  0.3443   |  99.16    |  1.664    |
|  3        |  0.8179   |  0.7307   |  735.7    |  69.7     |  0.2815   |  51.96    |  0.8286   |
|  4        |  0.8097   |  0.6656   |  920.6    |  83.52    |  0.8422   |  83.37    |  6.937    |
|  5        |  0.7529   |  5.195    |  851.0    |  53.71    |  0.03717  |  50.87    |  0.7373   |
|  6        |  0.8097   |  7.355    |  758.2    |  65.22    |  0.2815   |  99.86    |  0.9663   |
|  7        |  0.58     |  5.539    |  588.0    |  52.4     |  0.7306   |  39.05    |  2.804    |
|  8        |  0.8097   |  2.871    |  957.8    |  93.5     |  0.8157   |  13.07    |  6.604    |
|  9        |  0.796

## 5. Model optimization with layers optimization

In [216]:
# Create function
def nn_cl_bo2(neurons1, neurons2, activation1, activation2, optimizer, learning_rate, batch_size, epochs,layers1, 
              layers2, normalization, dropout1, dropout_rate1, dropout2, dropout_rate2):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD = {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    neurons1   = round(neurons1)
    neurons2   = round(neurons2)
    activation1 = activationL[round(activation1)]
    activation2 = activationL[round(activation2)]
    optimizer  = list(optimizerD.keys())[round(optimizer)]
    batch_size = round(batch_size)
    epochs     = round(epochs)
    layers1    = round(layers1)
    layers2    = round(layers2)
    
    score_acc = make_scorer(accuracy_score)
    
    def nn_cl_fun():        
        inputs  = keras.Input(shape=(144,33))
        x       = Flatten()(inputs)
        x       = Dense(neurons1, activation=activation1)(x)
        
        if normalization > 0.5:
            x = BatchNormalization()(x)     
            
        for i in range(layers1):
            x = Dense(neurons1, activation=activation1)(x)
        if dropout1 > 0.5:
            x = Dropout(dropout_rate1, seed=123)(x)
            
        for i in range(layers2):
            x = Dense(neurons2, activation=activation2)(x)
        if dropout2 > 0.5:
            x = Dropout(dropout_rate2, seed=123)(x)
        
        outputs = Dense(1, activation = activation2)(x)
        nn      = keras.Model(inputs, outputs)
        nn.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
        return nn

        nn.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
        return nn
    
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train_mio, y_train_mio, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [ ]:
params_nn2 ={
    'neurons1': (10, 100),
    'neurons2': (10, 100),
    'activation1':(0, 9),
    'activation2':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size':(200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout1':(0,1),
    'dropout_rate1':(0.1,0.5),
    'dropout2':(0,1),
    'dropout_rate2':(0.1,0.5)
}
# Run Bayesian Optimization
t = time.time()
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)
print("elapsed ", time.time()-t)

|   iter    |  target   | activa... | activa... | batch_... | dropout1  | dropout2  | dropou... | dropou... |  epochs   |  layers1  |  layers2  | learni... | neurons1  | neurons2  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.568    |  5.51     |  1.522    |  548.8    |  0.7693   |  0.2953   |  0.1597   |  0.109    |  53.62    |  1.477    |  1.675    |  0.9908   |  31.4     |  17.31    |  0.6696   |  4.349    |
|  2        |  0.683    |  2.468    |  4.196    |  294.7    |  0.07396  |  0.9008   |  0.4176   |  0.4362   |  85.22    |  2.982    |  2.155    |  0.8156   |  47.92    |  12.47    |  0.4541   |  0.7373   |
|  3        |  0.8097   |  7.355    |  6.28     |  652.2    |  0.2742   |  0.9985   |  0.1552   |  0.3462   |  58.8     |  1.81     |  2.456    |  0.3296   |  46.05    |  38.71

In [ ]:
params_nn_ = nn_bo.max['params']
learning_rate = params_nn_['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation1'] = activationL[round(params_nn_['activation1'])]
params_nn_['activation2'] = activationL[round(params_nn_['activation2'])]
params_nn_['batch_size'] = round(params_nn_['batch_size'])
params_nn_['epochs'] = round(params_nn_['epochs'])
params_nn_['layers1'] = round(params_nn_['layers1'])
params_nn_['layers2'] = round(params_nn_['layers2'])
params_nn_['neurons1'] = round(params_nn_['neurons1'])
params_nn_['neurons2'] = round(params_nn_['neurons2'])
params_nn_['normalization'] = round(params_nn_['normalization'])

optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),'Adadelta':Adadelta(lr=learning_rate),
             'Adagrad':Adagrad(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate),
             'Adamax':Adamax(lr=learning_rate),'Nadam':Nadam(lr=learning_rate)}
params_nn_['optimizer'] = list(optimizerD.keys())[round(params_nn_['optimizer'])]
params_nn_

In [76]:
len(X_data['admission_id'].unique().tolist())

7657

In [92]:
print(X_data[1:-1].columns.tolist()[1:-1])
len(X_data[1:-1].columns.tolist()[1:-1])

['rr', 'ews', 'heart_rate', 'temperature', 'sbp', 'dbp', 'Oxygen_Saturation', 'Assisted_O2', 'Confusion', 'CREA', 'UREA', 'K', 'GFR', 'WBC', 'PLT', 'HCT', 'HGB', 'RBC', 'MCH', 'MCV', 'NEUAB', 'TLYMAB', 'EOSAB', 'MONAB', 'BASAB', 'ALB', 'ALP', 'BILI', 'no_sample_series', 'sex', 'ethnicity', 'age_at_admin', 'Comorb_score', 'Spcfc_Comorb']


34

In [84]:
len(X_data[1:-1])

1102606

In [85]:
7657*144

1102608

In [82]:
7657*144*36

39693888

In [65]:
##################################################
##################################################
#  O. LOAD DATA AND FEATURE SELECTION
##################################################
t = time.time()
timeSeries      = True
peakstroughs    = False
samp_to_extract = 0
df_patients, df_admissions, df_eobs = load_fn.Load_data()
#X_data              = load_fn.Exctract_Xdata(df_patients, df_admissions, df_eobs, samp_to_extract, peakstroughs, timeSeries)
X_data= pickle.load(open('timeseries_X_data.pickle','rb'))
print("Elapsed time Loading Data:", time.time()-t)

Elapsed time Loading Data: 13.299950122833252


#### Normalisation

In [66]:
#-----------------------------------------------
# Feature Selection, 
# List of features creation
feat_list = X_data.columns.tolist()
feat_list = feat_list[1:-1]
feat_list = ['rr', 'ews', 'heart_rate', 'temperature', 'sbp', 'dbp', 'Oxygen_Saturation', 'Confusion', 'UREA', 
             'CREA', 'K', 'GFR', 'sex', 'ethnicity', 'age_at_admin', 'Comorb_score', 'Spcfc_Comorb']

In [67]:
##################################################
# PROPORTIONS OF TRAINING, VALIDATION AND TEST SET
##################################################
list_admins      = X_data['admission_id'].unique().tolist()
num_train_admins = int(0.5 * len(list_admins))
num_val_admins   = int(0.25 * len(list_admins))
num_test_admins  = len(list_admins) - num_train_admins - num_val_admins
print("num_train_samples:", num_train_admins)
print("num_val_samples:", num_val_admins)
print("num_test_samples:", num_test_admins)

num_train_samples: 4618
num_val_samples: 2309
num_test_samples: 2310


In [73]:
##################################################
#  NORMALISATION OF TRAINING SET
##################################################
raw_data  = X_data[feat_list]
mortality = X_data['Mortality'].tolist()

mean = raw_data[:num_train_admins * 144].mean(axis=0)
std  = raw_data[:num_train_admins * 144].std(axis=0)
std  = [1 if s == 0 else s for s in std]

norm_data = raw_data - mean
norm_data /= std

In [79]:
###################################################
# Training, Validation and test sets
###################################################

train_dataset = norm_data[:num_train_admins * 144]
val_dataset   = norm_data[num_train_admins * 144 : ((num_train_admins + num_val_admins) * 144)]
test_dataset  = norm_data[((num_train_admins + num_val_admins) * 144):]


# 2. Classification Using MiniRocket

In [80]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sktime.datasets import load_arrow_head  # univariate dataset
from sktime.transformations.panel.rocket import MiniRocket

In [81]:
X_train, y_train = load_arrow_head(split="test", return_X_y=True)
X_test, y_test = load_arrow_head(split="train", return_X_y=True)
print(X_train.shape, X_test.shape) 

(175, 1) (36, 1)


In [82]:

minirocket = MiniRocket() 
minirocket.fit(X_train)
X_train_transform = minirocket.transform(X_train)
X_train_transform.shape

(175, 9996)

In [62]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

In [63]:
X_test_transform = minirocket.transform(X_test)
classifier.score(X_test_transform, y_test)

0.9166666666666666

In [ ]:
# GAM
# https://multithreaded.stitchfix.com/blog/2015/07/30/gam/
# https://codeburst.io/pygam-getting-started-with-generalized-additive-models-in-python-457df5b4705f

In [ ]:
# https://towardsdatascience.com/minirocket-fast-er-and-accurate-time-series-classification-cdacca2dcbfa
#https://towardsdatascience.com/sktime-a-unified-python-library-for-time-series-machine-learning-3c103c139a55
#https://pypi.org/project/sktime/
#https://arxiv.org/pdf/1302.2277.pdf


In [45]:

from sktime.datasets import load_arrow_head
from sktime.classification.compose import ComposableTimeSeriesForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_arrow_head(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)
classifier = ComposableTimeSeriesForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)


0.8867924528301887

In [53]:
len(y_test)

53

In [55]:
X_test

,dim_0
168,0 -1.531721 1 -1.541291 2 -1.51496...
19,0 -1.918966 1 -1.894600 2 -1.87130...
55,0 -1.939935 1 -1.937687 2 -1.91122...
73,0 -1.813234 1 -1.825545 2 -1.81662...
170,0 -1.625142 1 -1.622988 2 -1.62606...
95,0 -1.828368 1 -1.839300 2 -1.80252...
115,0 -1.890214 1 -1.904523 2 -1.90426...
22,0 -1.754535 1 -1.777870 2 -1.75141...
25,0 -1.632037 1 -1.630116 2 -1.60748...
28,0 -2.004295 1 -1.985612 2 -1.95328...


In [31]:
print(X.iloc[0])

dim_0    0     -1.963009
1     -1.957825
2     -1.95614...
Name: 0, dtype: object


In [40]:
X.iloc[1][0]

0     -1.774571
1     -1.774036
2     -1.776586
3     -1.730749
4     -1.696268
         ...   
246   -1.639989
247   -1.678683
248   -1.729227
249   -1.775670
250   -1.789324
Length: 251, dtype: float64

In [41]:
X


,dim_0
0,0 -1.963009 1 -1.957825 2 -1.95614...
1,0 -1.774571 1 -1.774036 2 -1.77658...
2,0 -1.866021 1 -1.841991 2 -1.83502...
3,0 -2.073758 1 -2.073301 2 -2.04460...
4,0 -1.746255 1 -1.741263 2 -1.72274...
...,...
170,0 -1.625142 1 -1.622988 2 -1.62606...
171,0 -1.657757 1 -1.664673 2 -1.63264...
172,0 -1.603279 1 -1.587365 2 -1.57740...
173,0 -1.739020 1 -1.741534 2 -1.73286...
